<a href="https://colab.research.google.com/github/donw385/DS-Unit-4-Sprint-2-NLP/blob/master/module4-Word-Embeddings/LS_DS_424_Word_Embeddings_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Use Word2Vec to train your own model on a dataset.

1) **Optional** - Find your own dataset of documents to train you model on. You are going to need a lot of data, so it's probably not realistic to scrape data for this assignment given the time constraints that we're working under. Try to find a dataset that has > 5000 documents.

- If you can't find a dataset to use try this one: <https://www.kaggle.com/c/quora-question-pairs>

2) Clean/Tokenize the documents.

3) Vectorize the model using Word2Vec and explore the results using each of the following at least one time:

- your_model.wv.most_similar()
- your_model.wv.similarity()
- your_model.wv.doesn't_match()

In [2]:
!pip install kaggle


In [7]:
!pip install -U gensim
import gensim

     |████████████████████████████████| 24.2MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/


env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [6]:
!kaggle competitions download -c quora-question-pairs

  0% 0.00/4.95M [00:00<?, ?B/s]
100% 4.95M/4.95M [00:00<00:00, 79.4MB/s]
 43% 9.00M/21.2M [00:00<00:00, 32.5MB/s]
100% 21.2M/21.2M [00:00<00:00, 53.5MB/s]
 93% 105M/112M [00:01<00:00, 89.5MB/s] 
100% 112M/112M [00:01<00:00, 95.4MB/s]


In [8]:
!unzip train.csv.zip


Archive:  train.csv.zip
  inflating: train.csv               


In [14]:
!pip install modin

     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 10.1MB 47.2MB/s 
     |████████████████████████████████| 73.0MB 42.9MB/s 
     |████████████████████████████████| 13.9MB 20.4MB/s 
     |████████████████████████████████| 71kB 24.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Successfully uninstalled numpy-1.16.3
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2


In [1]:
import modin.pandas as pd
import re
import string
import nltk
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-03_00-48-24_452/logs.
Waiting for redis server at 127.0.0.1:29125 to respond...
Waiting for redis server at 127.0.0.1:14016 to respond...
Starting Redis shard with 10.0 GB max memory.
Starting the Plasma object store with 7.0 GB memory using /tmp.
'pattern' package not found; tag filters are not available for English


In [0]:
df = pd.read_csv('train.csv')

In [0]:
df = df.drop(columns=['id', 'qid1', 'qid2'])
df = df.dropna()

In [6]:
df.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
def cleanup(col):
    table = str.maketrans(string.punctuation,' '*32)
    stop_words = set(stopwords.words('english'))
#     lemmatizer = WordNetLemmatizer()
    cleaned_listings = []
    
    for row in col:
#         print(row)
        # Strip punctuation everywhere,
        # replacing it with spaces so that words separated only
        # by punctuation don't get smooshed together
        no_punctuation = row.translate(table)
    #     print("No Punctuation:", no_punctuation)
    #     print('------------------------------------------')

        # Tokenize by word
        tokens = word_tokenize(no_punctuation)
    #     print("Tokens:", tokens)
    #     print('------------------------------------------')

        # Make all words lowercase
        lowercase_tokens = [w.lower() for w in tokens]
    #     print("Lowercase:", lowercase_tokens)
    #     print('------------------------------------------')

        # Remove words that aren't alphabetic
        alphabetic = [w for w in lowercase_tokens if w.isalpha()]
    #     print("Alphabetic:", alphabetic)
    #     print('------------------------------------------')

        # Remove stopwords
        words = [w for w in alphabetic if not w in stop_words]
    #     print("Cleaned Words:", words)
    #     print("--------------------------------")

#         # lemmatize!
#         lemmas = [lemmatizer.lemmatize(w) for w in words]
        # Append to list
        cleaned_listings.append(words)
        
    return cleaned_listings

In [0]:
q1 = df.question1.tolist()
q2 = df.question2.tolist()


In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
q1_clean = cleanup(q1)


In [0]:
q2_clean = cleanup(q2)


In [0]:
df['q1_clean'] = q1_clean
df['q2_clean'] = q2_clean
all_qs = q1_clean + q2_clean

In [17]:

%%time
model1 = Word2Vec(all_qs, min_count=20, window=3, size=200)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 53464 words, keeping 11202 word types
PROGRESS: at sentence #20000, processed 107171 words, keeping 16008 word types
PROGRESS: at sentence #30000, processed 160656 words, keeping 19551 word types
PROGRESS: at sentence #40000, processed 213584 words, keeping 22394 word types
PROGRESS: at sentence #50000, processed 267253 words, keeping 25055 word types
PROGRESS: at sentence #60000, processed 320680 words, keeping 27280 word types
PROGRESS: at sentence #70000, processed 373981 words, keeping 29275 word types
PROGRESS: at sentence #80000, processed 427468 words, keeping 31006 word types
PROGRESS: at sentence #90000, processed 480867 words, keeping 32662 word types
PROGRESS: at sentence #100000, processed 534302 words, keeping 34257 word types
PROGRESS: at sentence #110000, processed 587129 words, keeping 35737 word types
PROGRESS: at sentence #120

CPU times: user 1min 26s, sys: 409 ms, total: 1min 26s
Wall time: 46.4 s


In [18]:
words = list(model1.wv.vocab)
print(len(words))
words

13899


['step',
 'guide',
 'invest',
 'share',
 'market',
 'india',
 'story',
 'kohinoor',
 'koh',
 'diamond',
 'increase',
 'speed',
 'internet',
 'connection',
 'using',
 'vpn',
 'mentally',
 'lonely',
 'solve',
 'one',
 'dissolve',
 'water',
 'sugar',
 'salt',
 'methane',
 'carbon',
 'di',
 'oxide',
 'astrology',
 'capricorn',
 'sun',
 'cap',
 'moon',
 'rising',
 'say',
 'buy',
 'good',
 'use',
 'instead',
 'motorola',
 'company',
 'hack',
 'charter',
 'method',
 'find',
 'separation',
 'read',
 'youtube',
 'comments',
 'make',
 'physics',
 'easy',
 'learn',
 'first',
 'sexual',
 'experience',
 'like',
 'laws',
 'change',
 'status',
 'student',
 'visa',
 'green',
 'card',
 'us',
 'compare',
 'immigration',
 'canada',
 'would',
 'trump',
 'presidency',
 'mean',
 'current',
 'international',
 'master',
 'students',
 'manipulation',
 'girls',
 'want',
 'friends',
 'guy',
 'reject',
 'many',
 'quora',
 'users',
 'posting',
 'questions',
 'readily',
 'answered',
 'google',
 'best',
 'digital',


In [19]:
model1.wv.similarity('cotton','addicted')


-0.00021207887

In [20]:
model1.wv.doesnt_match(['cotton','rabbit','shirt','polyester'])


precomputing L2-norms of word weight vectors


'rabbit'

In [21]:
model1.wv.most_similar('cotton', topn=10)


[('polyester', 0.8174766302108765),
 ('wool', 0.7866145372390747),
 ('denim', 0.7802823781967163),
 ('fabric', 0.7706536054611206),
 ('gin', 0.7656160593032837),
 ('stain', 0.755588948726654),
 ('trousers', 0.7486355900764465),
 ('jacket', 0.7470711469650269),
 ('coat', 0.7406226992607117),
 ('pepper', 0.7401096820831299)]

### Stretch Goals:

1) Use Doc2Vec to train a model on your dataset, and then provide model with a new document and let it find similar documents.

2) Download the pre-trained word vectors from Google. Access the pre-trained vectors via the following link: https://code.google.com/archive/p/word2vec

Load the pre-trained word vectors and train the Word2vec model

Examine the first 100 keys or words of the vocabulary

Outputs the vector representation for a select set of words - the words can be of your choice

Examine the similarity between words - the words can be of your choice

For example:

model.similarity('house', 'bungalow')

model.similarity('house', 'umbrella')